# Transformação para formato Time Series

## Pacotes necessários

In [4]:
import pandas as pd
import fastparquet
import os

## Leitura dos dados

In [10]:
# Carregar o dataframe recodificado

# Costela
#df = pd.read_parquet("/home/costela/Documentos/Escorpiao-series/Dados_processados/df1_recodificado.parquet")

# Usuario
df = pd.read_parquet("/home/usuario/Documentos/Escorpiao_series/Dados_processados/df1_recodificado.parquet")

# Filtrar os dados para incluir apenas os anos de 2015 a 2023
df = df[df['ANO'].isin(range(2014, 2023))]

## Agrupamento e calculo da Tx Incidencia

In [12]:
# Agrupar os dados por município, ano e população, contando o número de casos
dfg = df.groupby(['NOME_MUNI', 'ANO', 'MES', 'POPULACAO']).size().reset_index(name='CASOS')

# 6. Calcular taxa mensal por 100.000 habitantes
dfg['TAXA_MENSAL'] = (dfg['CASOS'] / dfg['POPULACAO']) * 100000
dfg['TAXA_MENSAL'] = dfg['TAXA_MENSAL'].round(1)

## Criar um dataframe por municipio

In [15]:
municipios = dfg['NOME_MUNI'].unique()
series_por_municipio = {}

for municipio in municipios:
    # Criar uma cópia selecionada para evitar SettingWithCopyWarning. Sem o copy, o objeto é so uma view do dataframe original, com o copy, é um novo objeto independente.    
    df_mun = dfg[dfg['NOME_MUNI'] == municipio].copy() 
    df_mun = df_mun.sort_values(['ANO','MES']) # organizar por ano
    df_mun['DATA'] = pd.to_datetime(df_mun['ANO'].astype(str) + '-' + df_mun['MES'].astype(str) + '-01') # definir o ano como índice (apropriado para séries temporais)
    df_mun.set_index('DATA', inplace=True)
    series_por_municipio[municipio] = df_mun[['TAXA_MENSAL']] # manter apenas a coluna de interesse

In [16]:
series_por_municipio['AGUAI']

,TAXA_MENSAL
DATA,
2014-01-01,12.2
2014-02-01,6.1
2014-06-01,12.2
2014-09-01,12.2
2014-10-01,6.1
...,...
2022-08-01,18.2
2022-09-01,30.4
2022-10-01,60.8
